In [1]:
import sys
sys.path.append('/vegas/apps/compiler/intel/intelpython3/bin')

import subprocess
nengines = 30 # or whatever
subprocess.Popen(["ipcluster", "start", "-n={:d}".format(nengines), "--profile=slurm"])

In [3]:
import ipyparallel
clients = ipyparallel.Client(profile='slurm')
print(clients.ids)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]


In [11]:
def testfun():
    print(123)
    import matplotlib.pyplot as plot
    plot.plot([1,2,3])

    return 456

In [19]:
clients[1].apply_sync(testfun)

<AsyncResult: testfun>

In [41]:
import glob
import os
import subprocess
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio

In [76]:
dir = '/vegas/scratch/womo1998/data_hadidi/datasets_linear_ambient/*.csv'
files = glob.glob(dir)
while True:
    snr_db=(np.random.rand(1)-0.5)*2*20 # -20 dB to 20 dB
    snr=10**(snr_db/10)
    print(snr_db,snr)
    #snr=np.infty
    file=files[np.random.randint(0,len(files))]
    
    array=np.loadtxt(file)
    
#     plt.figure()
    
#     plt.psd(array[:,1],Fs=array.shape[0]/(array[-1,0]-array[0,0]))
    power=np.mean(array[:,1]**2)

#     plt.axhline(10*np.log10(power), color='red')
#     ymin,ymax=plt.ylim()
#     ymin=ymax-0.25*(ymax-ymin)
#     plt.ylim(ymin=ymin)
    #decimate
    print(array[:12,0])
    print((array[-1,0]-array[0,0])/array.shape[0],6*(array[-1,0]-array[0,0])/array.shape[0])
    array=array[::6,:]
    print(array[:,0])
    print((array[-1,0]-array[0,0])/array.shape[0])
    power2=np.mean(array[:,1]**2)
    N=array.shape[0]
    # add noise
    noise_power = power/snr
    noise=np.random.normal(0,np.sqrt(noise_power),N)
    power_noise=np.mean(noise**2)
#     print(power,snr,np.sqrt(noise_power),noise)
    
    snr_actual=power/power_noise
    snr_actual_db= 10*np.log10(snr_actual)
    
#     print(snr_actual_db,snr_actual)
    
    array[:,1]+=noise
    
#     plt.figure()
#     plt.psd(array[:,1],Fs=array.shape[0]/(array[-1,0]-array[0,0]))
    
#     plt.psd(noise,Fs=array.shape[0]/(array[-1,0]-array[0,0]))
    
#     plt.axhline(10*np.log10(power), color='red')
#     plt.axhline(10*np.log10(power2), color='yellow', ls='dashed')
    
#     plt.axhline(10*np.log10(power_noise), color='green')
#     _,ymax=plt.ylim()
#     plt.ylim(ymax=max(ymax,10*np.log10(power),10*np.log10(power_noise)),ymin=ymin)
    cov = np.zeros(N)
    for i in range(N):
        cov[i]=array[:N-i,1].dot(array[i:,1])/(N-i)
    
#     plt.figure()
#     plt.plot(array[:,0],cov)
#     plt.show()
    
    mdict={'disp':array, 'corr':cov}
    
    sio.savemat('/vegas/scratch/womo1998/data_hadidi/test.mat', mdict, appendmat=True, format='5', long_field_names=False, do_compression=True, oned_as='row')
    break    


[16.02036337] [39.99782139]
[-6.93889390e-18  1.20193732e-02  2.40387464e-02  3.60581196e-02
  4.80774928e-02  6.00968660e-02  7.21162392e-02  8.41356125e-02
  9.61549857e-02  1.08174359e-01  1.20193732e-01  1.32213105e-01]
0.012019284440706586 0.07211570664423952
[-6.93889390e-18  7.21162392e-02  1.44232478e-01 ...  1.62730294e+03
  1.62737505e+03  1.62744717e+03]
0.07211304374002986


In [135]:
ind=np.arange(0,34)
t=ind/10
print(t,((t[-1]-t[0])/(len(t)-1)))
tdec=t[1::6]
print(tdec,((tdec[-1]-tdec[0])/(len(tdec)-1)))


[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2 1.3 1.4 1.5 1.6 1.7
 1.8 1.9 2.  2.1 2.2 2.3 2.4 2.5 2.6 2.7 2.8 2.9 3.  3.1 3.2 3.3] 0.09999999999999999
[0.1 0.7 1.3 1.9 2.5 3.1] 0.6
